In [4]:
import pandas as pd
import numpy as np
import os

1\. **Text files**

Perform the following operations on plain `txt` files:

+ create a list of integrer numbers and then save it to a text file named `data_int.txt`. Run the `cat` command to print the content of the file.
+ create a matrix of 5x5 floats and then save it to a text file named `data_float.txt`. Use the `cat` command to print the content of the file.
+ load the `txt` file of the previous point and convert it to a `csv` file by hand.

In [2]:
a=[x for x in range(10)]
file_name='./data_int.txt'

with open(file_name,'w') as f:
    f.write(str(a))

!cat data_int.txt

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [5]:
b=np.arange((25),dtype=float).reshape(5,5)
file_name='./data_float.txt'
with open(file_name,'w') as f:
  for i in range(5):
    for j in range(5):
      f.write(str(b[i,j])+' ')
    f.write('\n')
!cat data_float.txt

0.0 1.0 2.0 3.0 4.0 
5.0 6.0 7.0 8.0 9.0 
10.0 11.0 12.0 13.0 14.0 
15.0 16.0 17.0 18.0 19.0 
20.0 21.0 22.0 23.0 24.0 


In [6]:
matrix_5_5=np.loadtxt(file_name)

file_name='./data_float.csv'
with open(file_name,'w') as f:
  for i in range(5):
    for j in range(5):
      f.write(str(matrix_5_5[i,j])+',')
    f.write('\n')
  
!cat data_float.csv

0.0,1.0,2.0,3.0,4.0,
5.0,6.0,7.0,8.0,9.0,
10.0,11.0,12.0,13.0,14.0,
15.0,16.0,17.0,18.0,19.0,
20.0,21.0,22.0,23.0,24.0,


2\. **JSON files**

Load the file *user_data.json*, which can be found at:

- https://www.dropbox.com/s/sz5klcdpckc39hd/user_data.json

and filter the data by the "CreditCardType" when it equals to "American Express". Than save the data to a new CSV file.

In [7]:
!wget https://www.dropbox.com/s/sz5klcdpckc39hd/user_data.json

--2022-12-01 10:00:38--  https://www.dropbox.com/s/sz5klcdpckc39hd/user_data.json
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601f:18::a27d:912
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/sz5klcdpckc39hd/user_data.json [following]
--2022-12-01 10:00:38--  https://www.dropbox.com/s/raw/sz5klcdpckc39hd/user_data.json
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc0059b436631722778864b3cdb3.dl.dropboxusercontent.com/cd/0/inline/BxyoU3w8UvEwb0RCMDS6hBaqGS0sN5zMSCt1xsO9wCByXMKEpvzOzSYtyrbCYThbVDbnc_3aqQXDTQRpBlLKJ6JjyOWXnlAKhKqq6uENWfVbjZR60FOMlEwc9ldE5bvIYhY5Du45PXdiZNWcRdY7Yy70sWEvO08TE6I6jl-T8jZ7Bw/file# [following]
--2022-12-01 10:00:39--  https://uc0059b436631722778864b3cdb3.dl.dropboxusercontent.com/cd/0/inline/BxyoU3w8UvEwb0RCMDS6hBaqGS0sN5zMSCt1xsO9wCByXMKEpvzOzSYtyrbCYThbVDbnc_3aqQ

In [8]:
import json 
data = json.load(open('./user_data.json'))
print(data)

[{'ID': '1', 'JobTitle': 'Project Manager', 'EmailAddress': 'Remy_Crawford1065@typill.biz', 'FirstNameLastName': 'Remy Crawford', 'CreditCard': '6205-7258-7671-0612', 'CreditCardType': 'Capital One'}, {'ID': '2', 'JobTitle': 'Investment  Advisor', 'EmailAddress': 'Clint_Thorpe5003@bulaffy.com', 'FirstNameLastName': 'Clint Thorpe', 'CreditCard': '7083-8766-0251-2345', 'CreditCardType': 'American Express'}, {'ID': '3', 'JobTitle': 'Cook', 'EmailAddress': 'Russel_Hooper9403@bulaffy.com', 'FirstNameLastName': 'Russel Hooper', 'CreditCard': '6378-4461-8535-5847', 'CreditCardType': 'Discover'}, {'ID': '4', 'JobTitle': 'Business Broker', 'EmailAddress': 'Benjamin_Rowan6280@gembat.biz', 'FirstNameLastName': 'Benjamin Rowan', 'CreditCard': '5781-7635-1428-7414', 'CreditCardType': 'Discover'}, {'ID': '5', 'JobTitle': 'Loan Officer', 'EmailAddress': 'Matt_Lloyd3241@guentu.biz', 'FirstNameLastName': 'Matt Lloyd', 'CreditCard': '5771-8416-2736-1700', 'CreditCardType': 'Citibank'}, {'ID': '6', 'JobT

In [9]:
l=[]
for dic in data:
  if dic["CreditCardType"]=="American Express":
    l.append(dic)

df=pd.DataFrame(l)
df.to_csv('./user_data.csv')

3\. **CSV files with Pandas**

Load the file from this url:

- https://www.dropbox.com/s/kgshemfgk22iy79/mushrooms_categorized.csv

with Pandas. 

+ explore and print the DataFrame
+ calculate, using `groupby()`, the average value of each feature, separately for each class
+ save the file in a JSON format.

In [10]:
!wget https://www.dropbox.com/s/kgshemfgk22iy79/mushrooms_categorized.csv

--2022-12-01 10:01:41--  https://www.dropbox.com/s/kgshemfgk22iy79/mushrooms_categorized.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601b:18::a27d:812
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/kgshemfgk22iy79/mushrooms_categorized.csv [following]
--2022-12-01 10:01:41--  https://www.dropbox.com/s/raw/kgshemfgk22iy79/mushrooms_categorized.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uca6bbac38fba308a5aaec72c162.dl.dropboxusercontent.com/cd/0/inline/Bxxh5KrTQxQXnxHZKECwCcBjWUay_ru6EL054S54GygQNUIzCdGNbPHrGc5Pf6ap8RfsWJb9-zPQB_cBaO5B9S8kbOS1FYMaaz5Qa0gAAV670B_Zndy6Kr38DPsFpQ3p86Vw7rwyV5Q5MUFzxpdgl96lmt9EXi6sqYPhxHJwqE7_PA/file# [following]
--2022-12-01 10:01:42--  https://uca6bbac38fba308a5aaec72c162.dl.dropboxusercontent.com/cd/0/inline/Bxxh5KrTQxQXnxHZKECwCcBjWUay_ru6EL054S54G

In [11]:
df=pd.read_csv('mushrooms_categorized.csv')
df.head()

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,1,5,2,4,1,6,1,0,1,4,...,2,7,7,0,2,1,4,2,3,5
1,0,5,2,9,1,0,1,0,0,4,...,2,7,7,0,2,1,4,3,2,1
2,0,0,2,8,1,3,1,0,0,5,...,2,7,7,0,2,1,4,3,2,3
3,1,5,3,8,1,6,1,0,1,5,...,2,7,7,0,2,1,4,2,3,5
4,0,5,2,3,0,5,1,1,0,4,...,2,7,7,0,2,1,0,3,0,1


In [13]:
df1=df.groupby(['class']).mean()
df1

,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
class,,,,,,,,,,,,,,,,,,,,,
0,3.266160,1.615970,4.581749,0.653992,4.334601,0.954373,0.285171,0.068441,6.622624,0.615970,...,1.798479,6.098859,6.064639,0.0,1.931559,1.125475,3.007605,3.201521,3.283270,1.148289
1,3.436159,2.055158,4.421859,0.159346,3.940756,0.995403,0.028601,0.567926,2.863636,0.514811,...,1.394280,5.512768,5.504597,0.0,2.002043,1.009193,1.522983,4.021450,4.031665,1.895812


In [14]:
df1.to_json('mushrooms_categorized_means.json')
!cat mushrooms_categorized_means.json

{"cap-shape":{"0":3.2661596958,"1":3.4361593463},"cap-surface":{"0":1.6159695817,"1":2.0551583248},"cap-color":{"0":4.5817490494,"1":4.4218590398},"bruises":{"0":0.6539923954,"1":0.1593462717},"odor":{"0":4.3346007605,"1":3.9407558733},"gill-attachment":{"0":0.9543726236,"1":0.9954034729},"gill-spacing":{"0":0.2851711027,"1":0.0286006129},"gill-size":{"0":0.0684410646,"1":0.5679264556},"gill-color":{"0":6.6226235741,"1":2.8636363636},"stalk-shape":{"0":0.6159695817,"1":0.5148110317},"stalk-root":{"0":1.4980988593,"1":0.6925434116},"stalk-surface-above-ring":{"0":1.7756653992,"1":1.3595505618},"stalk-surface-below-ring":{"0":1.7984790875,"1":1.3942798774},"stalk-color-above-ring":{"0":6.0988593156,"1":5.5127681307},"stalk-color-below-ring":{"0":6.0646387833,"1":5.5045965271},"veil-type":{"0":0.0,"1":0.0},"veil-color":{"0":1.9315589354,"1":2.0020429009},"ring-number":{"0":1.1254752852,"1":1.0091930541},"ring-type":{"0":3.0076045627,"1":1.5229826353},"spore-print-color":{"0":3.2015209125,

4\. **Reading the credit card numbers**

Get the binary file named *credit_card.dat* from this address:

- https://www.dropbox.com/s/8m0syw2tkul3dap/credit_card.dat

and convert the data into the real credit card number, knowing that:
- each line corresponds to a credit card number, which consists of 16 characters (which are numbers in the 0-9 range) divided in 4 blocks, with a whitespace between each block
- each character is written using a 6 bit binary representation (including the whitespace)
- the final 4 bits of each line are a padding used to determine the end of the line, and can be ignored

*Hint*: convert the binary numbers to the decimal representation first, and then use the `chr()` function to convert the latter to a char

In [15]:
!wget https://www.dropbox.com/s/8m0syw2tkul3dap/credit_card.dat

--2022-12-01 10:02:59--  https://www.dropbox.com/s/8m0syw2tkul3dap/credit_card.dat
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/8m0syw2tkul3dap/credit_card.dat [following]
--2022-12-01 10:03:00--  https://www.dropbox.com/s/raw/8m0syw2tkul3dap/credit_card.dat
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc1054f764be0cf7350850822281.dl.dropboxusercontent.com/cd/0/inline/BxyCZgDsycsPjMgtGYYwKagpj9WxSI7H4R7rJadmGZPBz1zNwZtMtAHGdQIBf-XESEXk_jklTyv1Zqq21EOXkQnT51mtywvGVHaLRHUNSkomvXfQvgXb00YXD_ZdDStwUtptyO_YADkSjEC4bCVPy6_IP23oyAXXOfgc6AJ5hw4fNg/file# [following]
--2022-12-01 10:03:00--  https://uc1054f764be0cf7350850822281.dl.dropboxusercontent.com/cd/0/inline/BxyCZgDsycsPjMgtGYYwKagpj9WxSI7H4R7rJadmGZPBz1zNwZtMtAHGdQIBf-XESEXk_jk

In [16]:
Credit_cards=[]
with open('./credit_card.dat', 'r') as file:
  for line in file:
    s=''
    for i in range(0,len(line),6):
      s+=chr(int(line[i:i+6],2))
    Credit_cards.append(s[:-1])

print('List of credit cards:',Credit_cards)


List of credit cards: ['7648 5673 3775 2271', '3257 8247 3354 2266', '2722 0001 4011 6652', '0661 3063 3742 3150', '0432 1608 1462 4742', '5827 2027 8785 7303', '5774 8528 2087 1117', '8140 1210 6352 2845', '5764 1133 7301 7100', '6456 1737 4126 6726', '1228 8631 7382 0000', '7051 0160 5374 3166', '0618 3587 1630 6376', '1545 5454 7444 5636', '6735 3116 3202 6834', '7287 5011 1547 8413', '7033 2607 3328 4200', '2568 5244 1874 5024', '1684 2253 7570 7118', '0672 2576 0575 6631', '6332 8353 8787 1340', '1813 3361 1175 4211', '2477 6450 8840 2368', '5512 3505 2563 1326', '3083 7882 0621 0025', '4521 5148 8045 0334', '7563 3654 8713 5787', '8324 2664 0476 5561', '0565 2504 7168 3510', '5107 5507 1767 0738', '2462 1821 2448 1443', '2788 0638 6861 6554', '5851 5873 5474 0547', '0670 1004 4013 2655', '5874 5506 3048 0806', '2805 5401 8462 1260', '5083 8406 6310 1862', '1076 1445 3013 2266', '8440 4804 4844 5277', '4758 6141 0686 1387', '7586 0675 0315 2568', '2544 1258 7432 5165', '3474 5023 

5\. **Write data to a binary file**

a) Start from the `data/data_000637.txt` file that we have used during the previous lectures, and convert it to a binary file according to the format defined below:

In [18]:
#from IPython.display import Image
#Image("images/data_format.png")

*Hints*:
- Read the first 10 lines using Pandas
- Iterate over the DataFrame rows
- For every row, ``pack'' the values (features) into a single 64-bit word, according to the format specified above. Use bit-wise shifts and operators to do so.
- Write each 64-bit word to a binary file. You can use `struct` in this way:
```
binary_file.write( struct.pack('<q', word) )
```
where `word` is the 64-bit word.
- Close the file after completing the loop.


In [19]:
!wget https://www.dropbox.com/s/ga9wi6b40cakgae/data_000637.txt

--2022-12-01 10:03:45--  https://www.dropbox.com/s/ga9wi6b40cakgae/data_000637.txt
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/ga9wi6b40cakgae/data_000637.txt [following]
--2022-12-01 10:03:45--  https://www.dropbox.com/s/raw/ga9wi6b40cakgae/data_000637.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucfffde37b5ee9c39d3493458bf2.dl.dropboxusercontent.com/cd/0/inline/BxxRY97jSKegvGAJOxIMbM9B60ANg1ItKW4_9g8udgUeCGNDFcK-MOfdHgbFTF8bYNTf_UxC7UhYo5uZb34JQP4PXRUb1H092aItyDuWn1Fng16OVTFO6vBfz4er-PovGzzEyL-HF3uxjVsLYbRI4MWksWVDDEv5oemi5tnW0kqiEA/file# [following]
--2022-12-01 10:03:45--  https://ucfffde37b5ee9c39d3493458bf2.dl.dropboxusercontent.com/cd/0/inline/BxxRY97jSKegvGAJOxIMbM9B60ANg1ItKW4_9g8udgUeCGNDFcK-MOfdHgbFTF8bYNTf_Ux

In [24]:
data1=pd.read_csv('./data_000637.txt',nrows=10)
data1

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS
0,1,0,123,3869200167,2374,26
1,1,0,124,3869200167,2374,27
2,1,0,63,3869200167,2553,28
3,1,0,64,3869200167,2558,19
4,1,0,64,3869200167,2760,25
5,1,0,63,3869200167,2762,4
6,1,0,61,3869200167,2772,14
7,1,0,139,3869200167,2776,0
8,1,0,62,3869200167,2774,21
9,1,0,60,3869200167,2788,7


In [26]:
import struct
file_name = "data_000637.dat"

with open(file_name, 'wb') as binary_file:
  for line in data1.values: 
    word = (line[5] << 0 ) 
    word += (line[4] << 5 ) 
    word += (line[3] << 17) 
    word += (line[2] << 49) 
    word += (line[1] << 58)
    word += (line[0] << 62)

    binary_file.write( struct.pack('<q', word))
binary_file.close()


b) Check that the binary file is correctly written by reading it with the code used in the lecture `06_OSEMN.ipynb`, and verify that the content of the `txt` and binary files is consistent.


In [27]:
import  time

data = {}
columns = ['HEAD', 'FPGA', 'CHANNEL', 'ORBIT_CNT', 'BX_CNT', 'TDC_MEAS']
df = pd.DataFrame({}, columns=columns)

with open('./data_000637.dat', 'rb') as file:
    file_content = file.read()
    word_counter = 0
    word_size = 8 # size of the word in bytes
    for i in range(0, len(file_content), word_size):
        word_counter += 1
        if word_counter > 10: break
        word = struct.unpack('<q', file_content[i : i + word_size])[0] # get an 8-byte word
        head     = (word >> 62) & 0x3
        fpga     = (word >> 58) & 0xF
        tdc_chan = (word >> 49) & 0x1FF
        orb_cnt  = (word >> 17) & 0xFFFFFFFF
        bx       = (word >> 5 ) & 0xFFF
        tdc_meas = (word >> 0 ) & 0x1F
        #if i == 0: print ('{0}\t{1}\t{2}\t{3}\t{4}\t{5}'.format('HEAD', 'FPGA', 'CHANNEL', 'ORBIT_CNT', 'BX_CNT', 'TDC_MEAS'))
        #print('{0}\t{1}\t{2}\t{3}\t{4}\t{5}'.format(head, fpga, tdc_chan, orb_cnt, bx, tdc_meas))
        entry = {'HEAD' : head, 'FPGA' : fpga, 'CHANNEL' : tdc_chan, 'ORBIT_CNT' : orb_cnt, 'BX_CNT' : bx, 'TDC_MEAS' : tdc_meas}
        df = df.append(entry, ignore_index=True)

df

,HEAD,FPGA,CHANNEL,ORBIT_CNT,BX_CNT,TDC_MEAS
0,1,0,123,3869200167,2374,26
1,1,0,124,3869200167,2374,27
2,1,0,63,3869200167,2553,28
3,1,0,64,3869200167,2558,19
4,1,0,64,3869200167,2760,25
5,1,0,63,3869200167,2762,4
6,1,0,61,3869200167,2772,14
7,1,0,139,3869200167,2776,0
8,1,0,62,3869200167,2774,21
9,1,0,60,3869200167,2788,7


The content of the txt and binary files is consistent.


c) What is the difference of the size on disk between equivalent `txt` and binary files?

In [28]:
data1.iloc[0:10,:].to_csv('./data_000637_1.txt')

In [29]:
size_txt=os.path.getsize("./data_000637_1.txt") #os.path.getsize() returns the size of the file in bytes
size_bin=os.path.getsize("./data_000637.dat")

In [30]:
print('The size of the text file is:',size_txt)
print('The size of the binary file is:',size_bin)

The size of the text file is: 333
The size of the binary file is: 80


The text file is much larger than the binary file and that's because in the text file, text, character, numbers are stored one character per byte i.e. 32667 occupies 5 bytes even though it occupies 2 bytes in memory.
However, in binary file, data is stored in binary format and each data would occupy the same number of bytes on disks as it occupies in memory.